In [12]:
import pandas as pd

# Baca data
fact = pd.read_csv("fact_table.csv")

# Pastikan kolom waktu jadi datetime
fact['FaultArisingTime'] = pd.to_datetime(fact['FaultArisingTime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
fact['FaultClearingTime'] = pd.to_datetime(fact['FaultClearingTime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Hitung ulang Duration (dalam jam)
fact['Duration'] = (fact['FaultClearingTime'] - fact['FaultArisingTime']).dt.total_seconds() / 3600

# Hapus nilai durasi negatif (jika ada anomali)
fact = fact[fact['Duration'] >= 0]

# Bulatkan hasil durasi biar rapi (2 angka di belakang koma)
fact['Duration'] = fact['Duration'].round(2)

# Cek hasil
print(fact[['FaultArisingTime', 'FaultClearingTime', 'Duration']].head())


     FaultArisingTime   FaultClearingTime  Duration
0 2025-06-30 23:28:00 2025-07-02 08:30:19     33.04
1 2025-06-30 23:27:00 2025-06-30 23:33:27      0.11
2 2025-06-30 23:07:00 2025-07-01 17:02:39     17.93
3 2025-06-30 22:49:00 2025-06-30 23:03:58      0.25
4 2025-06-30 22:22:00 2025-07-02 10:33:41     36.19


In [13]:
with open("fact_table_final.csv", 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i == 5265:
            print(line)
            break


OGV2503130076;CiSO - CIM Service;Medium;Fault;REG Medium (VSO) - Service degradation;Closed;-;Outgoing Voice;13/03/2025 18:59;16/03/2025 09:13;P3;62.25;2;6;4



In [14]:
# Membaca semua file CSV dengan pemisah ;
fact = pd.read_csv("fact_table_final.csv", sep=';')
dim_status = pd.read_csv("dim_status.csv", sep=';')
dim_service = pd.read_csv("dim_service.csv", sep=';')
dim_ticket = pd.read_csv("dim_ticket.csv", sep=';')
dim_user = pd.read_csv("dim_user.csv", sep=';')

# Tampilkan 5 baris pertama dari fact table untuk memastikan sudah benar
print(fact.head())

          Number                           creator Severity TicketType  \
0  VPN2506300209  CISO - Connectivity Surveillance     High      Fault   
1  VPN2506300208  CISO - Connectivity Surveillance     High      Fault   
2  IPT2506300206                CiSO - CIM Service   Medium    Request   
3  IPL2506300204                CiSO - CIM Service     High      Fault   
4  HBV2506300202                CiSO - CIM Service   Medium      Fault   

                                     Request/Fault  Status ServiceType  \
0           VVIP High (CSO) -  Single circuit down  Closed     Revenue   
1           VVIP High (CSO) -  Single circuit down  Closed     Revenue   
2  VVIP Request (CSO) – Request SIP/LOG/RFO/Adv IP  Closed           -   
3             VIP High (CSO) -  Single ciruit down  Closed     Revenue   
4           REG Medium (VSO) - Service degradation  Closed           -   

    ProductGroup  FaultArisingTime FaultClearingTime Priority  Duration  \
0         IP VPN  30/06/2025 23:28 

In [15]:
print(dim_status.columns)
print(fact.columns)


Index(['Status', 'Status_ID'], dtype='object')
Index(['Number', 'creator', 'Severity', 'TicketType', 'Request/Fault',
       'Status', 'ServiceType', 'ProductGroup', 'FaultArisingTime',
       'FaultClearingTime', 'Priority', 'Duration', 'User_ID', 'Service_ID',
       'Ticket_Type_ID'],
      dtype='object')


In [17]:
dim_status = pd.read_csv("dim_status.csv", sep=",")
print(dim_status.head())
print(dim_status.columns)


       Status;Status_ID
0              Closed;1
1    Technical Closed;2
2                Open;3
3             Testing;4
4  Notifying Customer;5
Index(['Status;Status_ID'], dtype='object')


In [18]:
dim_status = pd.read_csv("dim_status.csv", sep=';')
print(dim_status.head())
print(dim_status.columns)


               Status  Status_ID
0              Closed          1
1    Technical Closed          2
2                Open          3
3             Testing          4
4  Notifying Customer          5
Index(['Status', 'Status_ID'], dtype='object')


In [19]:
fact['Status_ID'] = fact['Status'].map(
    dict(zip(dim_status['Status'], dim_status['Status_ID']))
)

print(fact[['Status', 'Status_ID']].head())


   Status  Status_ID
0  Closed          1
1  Closed          1
2  Closed          1
3  Closed          1
4  Closed          1


In [ ]:
# Simpan ke file baru
fact.to_csv("fact_tables.csv", index=False)

✅ Kolom Duration sudah diperbaiki dan disimpan ke fact_table_final.csv
